# Analyse univariée

In [1]:
import pandas as pd
import traitement_texte as tt
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
os.getcwd()

'/home/bryan/OpenClassRoom/Projet_Nutri/netoyage données'

In [3]:
nom_fichier = 'df_clean.csv'
PATH_DIRECTORY = '/home/bryan/OpenClassRoom/Projet_1/data'
PATH_COMPLET = os.path.join(PATH_DIRECTORY, nom_fichier)
df = pd.read_csv(PATH_COMPLET, index_col = 0, low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/bryan/OpenClassRoom/Projet_1/data/df_clean.csv'

In [ ]:
graph_path = '/home/bryan/OpenClassRoom/Projet_1/graphique'

## analyse de la variable "additives"

In [ ]:
df['additives_n'].unique()

In [ ]:
df['additives_tags'].unique()

In [ ]:
df['additives_tags'].isna().sum()

In [ ]:
def transform_to_list(string):
    string = str(string).split(" ")
    return string

In [ ]:
df['additives_tags'][df['additives_tags'].notna()] = df['additives_tags'].apply(transform_to_list)

In [ ]:
df['additives_tags'][df['additives_tags'].notna()]

In [ ]:
# creation champ  'nombre additif'
df['nbre_additifs']=0
#creation d'une liste contenant les additifs
liste_additifs = []
for index, ligne in enumerate(df['additives_tags'][df['additives_tags'].notna()]):
    #on recupère le nombre d'additif dans df['nbre_additifs']
    df['nbre_additifs'].loc[index] = len(ligne)
    #on ajoute l'additif dans un dictionanire
    for count, value in enumerate(ligne):
         #on ajoute l'additif dans la liste_additifs:
        if ligne[count] not in liste_additifs:
            liste_additifs.append(ligne[count])

In [ ]:
liste_additifs

In [ ]:
#creation d'un dictionnaire avec pour key = nom additif, et pour value = nombre d'occurence
dict_additifs = {}
for nom_additif in liste_additifs:
    nbre_occurence = 0
    for ligne in df['additives_tags'][df['additives_tags'].notna()]:
        if nom_additif in ligne:
            nbre_occurence += 1
    dict_additifs[nom_additif] = nbre_occurence

In [ ]:
dict_additifs

In [ ]:
#transformation du dict_additifs en DF
df_additifs = pd.DataFrame(list(dict_additifs.items()),columns=['additif', 'occurence'])
df_additifs.head(10)

In [ ]:
df_additifs = df_additifs.sort_values(by =['occurence'], axis=0, ascending=False, inplace=False)
df_additifs.head()

In [ ]:
#selection des additifs les plus present
X = []
Y = []
for cle in dict_additifs:
    if dict_additifs[cle] >4000 :
        X.append(cle)
        Y.append(dict_additifs[cle])

In [ ]:
sns.set_theme(palette="pastel")
plt.figure(figsize=(14, 6))
ax = sns.barplot(x=Y, y=X, data=df_additifs, orient='horizontal')
plt.title("Principaux additifs", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Principaux additifs.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

In [ ]:
#l additif le plus present est 'e330': acide citrique

In [ ]:
#recherche de corrélation entre le nombre d'additif et le nutrition grade ou score

In [ ]:
df.groupby(['nutrition_grade_fr'])['nbre_additifs'].mean()

In [ ]:
sns.set_theme(palette="pastel")
plt.figure(figsize=(14, 6))
sns.boxplot(x='nutrition_grade_fr', y='nbre_additifs', data=df)

plt.title("Nombre additifs & Nutrition grade", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Nombre additifs & Nutrition grade.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

In [ ]:
df.groupby(['nbre_additifs'])['nutrition-score-uk_100g'].mean()

In [ ]:
sns.set_theme(palette="pastel")
plt.figure(figsize=(14, 6))
sns.scatterplot(data=df, x='nutrition-score-uk_100g', y='nbre_additifs')

plt.title("Nombre additifs & Nutrition-score", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Nombre additifs & Nutrition-score.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

### pas d'influence entre le nombre d'additifs et le le nutrution grade.

# analyse de la variable 'nutrition_grade_fr' & 'nutrition-score-uk_100g'

In [ ]:
nutrition_grade = df.groupby(by = 'nutrition_grade_fr')
df_nutrition_grade = df[['nutrition_grade_fr','nutrition-score-uk_100g']].sort_values(by = 'nutrition_grade_fr' )

In [ ]:
#Distribution nutrition grade
sns.set_theme(palette="pastel")
plt.figure(figsize=(8, 6))
sns.histplot(df_nutrition_grade, x='nutrition_grade_fr', shrink=0.9)

plt.title("Distribution nutrition grade", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Distribution nutrition grade.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')


In [ ]:
#Distribution nutrition-score
sns.set_theme(palette="pastel")
plt.figure(figsize=(14, 6))
sns.histplot(data = df['nutrition-score-uk_100g'], kde = True, bins = 50)

plt.title("Distribution nutrition-score", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Distribution nutrition-score.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

In [ ]:
#Test de normalité sur la distribution du champ
a = df['nutrition-score-uk_100g']
alpha = 0.05
k2, p = stats.normaltest(a=a, axis=0, nan_policy='omit')
print("p = {:g}".format(p))

# null hypothesis H0: x comes from a normal distribution
if p < alpha: 
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")

In [ ]:
#Verification qu il existe bien une corelation entre le score et le grade:
sns.set_theme(palette="pastel")
plt.figure(figsize=(8, 6))
sns.boxplot(x='nutrition_grade_fr', y='nutrition-score-uk_100g', data=df_nutrition_grade)

plt.title("Nutrition Grade - Nutrition-score", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Nutrition Grade - Nutrition-score - boxplot.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

### Les propriétés asymptotiques de l'ANOVA signifient que plus on a de données à disposition, plus l'ANOVA est capable de supporter un écart important aux hypothèses

In [ ]:
from scipy.stats import f_oneway

In [ ]:
df_a = nutrition_grade.get_group('a')
df_b = nutrition_grade.get_group('b')
df_c = nutrition_grade.get_group('c')
df_d = nutrition_grade.get_group('d')
df_e = nutrition_grade.get_group('e')

In [ ]:
a = df_a['nutrition-score-uk_100g'].values
b = df_b['nutrition-score-uk_100g'].values
c = df_c['nutrition-score-uk_100g'].values
d = df_d['nutrition-score-uk_100g'].values
e = df_e['nutrition-score-uk_100g'].values

In [ ]:
# null hypothesis H0: the population median of all of the groups are equal
if p < alpha: 
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")
f_oneway(a,b,c,d,e)

# Partie modifiée suite a la soutenance

In [ ]:
#Kruskal-Wallis H-test
# null hypothesis H0: the population median of all of the groups are equal
if p < alpha: 
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")
stats.kruskal(a,b,c,d,e)

### conclusion, il y a bien une une corellation entre le grade et le score

# analyse via les groupes alimentaires

In [ ]:
nutrition_grade = df.groupby(by = 'nutrition_grade_fr')
df_nutrition_grade = df[['nutrition_grade_fr','pnns_groups_1', 'pnns_groups_2','nutrition-score-uk_100g']].sort_values(by = ['nutrition_grade_fr','pnns_groups_1'])
df_pnns = df_nutrition_grade[df['pnns_groups_1'] != 'unknown']
df_pnns.shape

In [ ]:
sns.set_theme()

fig, ax = plt.subplots(1,1,figsize=(14, 6))

sns.countplot(x='pnns_groups_1',data =df_pnns, ax = ax)
fig.suptitle("Distribution groupe alimentaire N1", fontsize=20, color='tab:blue', fontweight="bold", y =0.92)
plt.ylabel("Count", size = 15,color='tab:blue')
plt.xlabel("Groupe alimentaire", size = 15,color='tab:blue')
ax.tick_params(pad = 0, labelsize=15, colors='tab:purple')
plt.xticks(rotation=90)

for container in ax.containers:
        ax.bar_label(container, padding=1)
        
path_complet = os.path.join(graph_path,'Distribution groupe alimentaire N1.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
sns.set_theme()
fig, ax = plt.subplots(1,1,figsize=(14, 6))
                   
sns.countplot(x='pnns_groups_2',data =df_pnns)

fig.suptitle("Distribution groupe alimentaire N2", fontsize=20, color='tab:blue', fontweight="bold", y=0.92)
plt.ylabel("Count", size = 15,color='tab:blue')
plt.xlabel("Groupe alimentaire", size = 15,color='tab:blue')
ax.tick_params(pad = 0, labelsize=15, colors='tab:purple')
plt.xticks(rotation=90)

for container in ax.containers:
        ax.bar_label(container, padding=1)                  
                          
path_complet = os.path.join(graph_path,'Distribution groupe alimentaire N2.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
sns.set_theme()
fig, ax =plt.subplots(figsize=(14, 6))

ax = sns.histplot(x='pnns_groups_1',data =df_pnns, hue ='nutrition_grade_fr', shrink=0.9)

plt.title("Repartition des grades nutritionnelle par groupe alimentaire N1", fontsize=20, color='tab:blue', fontweight="bold")
plt.ylabel("Count", size = 15,color='tab:blue')
plt.xlabel("Groupe alimentaire", size = 15,color='tab:blue')
ax.tick_params(pad = 0, labelsize=15, colors='tab:purple')
plt.xticks(rotation=90)

path_complet = os.path.join(graph_path,'Repartition des grades nutritionnelle par groupe alimentaire N1.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')
                
plt.show()

In [ ]:
sns.set_theme()
fig, ax = plt.subplots(1,1 ,figsize=(14, 6))

ax = sns.histplot(x='pnns_groups_2',data =df_pnns, hue ='nutrition_grade_fr', shrink=0.9)

plt.title("Repartition des grades nutritionnelle par groupe alimentaire N2", fontsize=20, color='tab:blue', fontweight="bold")
plt.ylabel("Count", size = 15,color='tab:blue')
plt.xlabel("Groupe alimentaire", size = 15,color='tab:blue')
ax.tick_params(pad = 0, labelsize=15, colors='tab:purple')
plt.xticks(rotation=90)

path_complet = os.path.join(graph_path,'Repartition des grades nutritionnelle par groupe alimentaire N2.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')
      

In [ ]:
#tableau de contingences entre pnns-groups_1 et nutritions grade
X = df_pnns['pnns_groups_1']
Y = df_pnns['nutrition_grade_fr']
data_crosstab = pd.crosstab(X, Y,dropna =True)
df_crosstab = pd.DataFrame(data_crosstab)
df_crosstab

In [ ]:
fig = plt.figure(num = None, figsize = (40,10), dpi = 80, facecolor ='w', edgecolor ='k')
plt.clf()
ax = fig.add_subplot(111)
ax.set_aspect(1)
res = sns.heatmap(df_crosstab, annot = False, fmt='.2f', cmap="YlGnBu", vmin=0.0, vmax =3000)

plt.title("Tableau de contingence : grades nutritionnelle Vs groupe alimentaire N1", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Tableau de contingence : grades nutritionnelle Vs groupe alimentaire N1.jpg')
fig.savefig(path_complet, dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# V de cramer
from scipy.stats.contingency import association
association(df_crosstab, method="cramer")

In [ ]:
#Le V de cramer > 0.30 on peut considerer que la relation entre pnns_groups_1 et le grade est forte.

In [ ]:
# le T de "tschuprow"
association(df_crosstab,method="tschuprow")

In [ ]:
#Verification qu il existe bien une corelation entre le score et le groupe alimentaire N1:
sns.set_theme()
fig, ax = plt.subplots(1,1,figsize=(14, 6))

ax = sns.boxplot(x='pnns_groups_1', y='nutrition-score-uk_100g', data=df_nutrition_grade.sort_values(by='nutrition-score-uk_100g'))

plt.title("correlation entre nutri-score et groupe alimentaire N1", fontsize=20, color='tab:blue', fontweight="bold")


plt.ylabel("Nutri-score", size = 15,color='tab:blue')
plt.xlabel("Groupe alimentaire", size = 15,color='tab:blue')
ax.tick_params(pad = 0, labelsize=15, colors='tab:purple')
plt.xticks(rotation=90)

path_complet = os.path.join(graph_path,'correlation entre score nutritionnelle et groupe alimentaire N1 - boxplot.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
X = df_nutrition_grade.groupby(['pnns_groups_1'])['nutrition-score-uk_100g'].agg('mean').sort_values()
col_pnns_group_1 = df_nutrition_grade.groupby(['pnns_groups_1']).groups.keys()
df_pnns_groups_1_mean=pd.DataFrame(X,col_pnns_group_1)
df_pnns_groups_1_mean = df_pnns_groups_1_mean.sort_values(by='nutrition-score-uk_100g')
df_pnns_groups_1_mean

In [ ]:
sns.set_theme()

fig , ax = plt.subplots(1,1, figsize =(14,6))

sns.barplot(y='nutrition-score-uk_100g',x =df_pnns_groups_1_mean.index, data=df_pnns_groups_1_mean, ax =ax)

fig.suptitle("score nutritionnel moyen par groupe alimentaire N1", fontsize=20, color='tab:blue', fontweight="bold")
plt.ylabel("Nutri-score", size = 20,color='tab:blue')
plt.xlabel("Groupe alimentaire", size = 15,color='tab:blue')
ax.tick_params(pad = 0, labelsize=15, colors='tab:purple')
plt.xticks(rotation = 90)

for container in ax.containers:
        ax.bar_label(container, padding=1, fmt = '%.2f')
        
path_complet = os.path.join(graph_path,'nutri-score moyen par groupe alimentaire N1.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')


plt.show()


In [ ]:
X = df_nutrition_grade.groupby(['pnns_groups_2'])['nutrition-score-uk_100g'].agg('mean').sort_values()
col_pnns_group_2 = df_nutrition_grade.groupby(['pnns_groups_2']).groups.keys()
df_pnns_groups_2_mean=pd.DataFrame(X,col_pnns_group_2)
df_pnns_groups_2_mean = df_pnns_groups_2_mean.sort_values(by='nutrition-score-uk_100g')
df_pnns_groups_2_mean

In [ ]:
sns.set_theme()
fig, ax = plt.subplots(1,1,figsize=(14, 6))
sns.barplot(y='nutrition-score-uk_100g',x =df_pnns_groups_2_mean.index, data=df_pnns_groups_2_mean)

fig.suptitle("nutri-score moyen par groupe alimentaire N2", fontsize=20, color='tab:blue', fontweight="bold")
plt.ylabel("Nutri-score", size = 20,color='tab:blue')
plt.xlabel("Groupe alimentaire", size = 15,color='tab:blue')
ax.tick_params(pad = 0, labelsize=15, colors='tab:purple')
plt.xticks(rotation = 90)

for container in ax.containers:
        ax.bar_label(container, padding=2, fmt='%.0f')
        
path_complet = os.path.join(graph_path,'nutri-score moyen par groupe alimentaire N2.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')


plt.show()


In [ ]:
#tableau de contingences entre pnns-groups_1 et nutritions grade
X = df_pnns['pnns_groups_2']
Y = df_pnns['nutrition_grade_fr']
data_crosstab = pd.crosstab(X, Y,dropna =True)
df_crosstab = pd.DataFrame(data_crosstab)
df_crosstab

In [ ]:
fig = plt.figure(num = None, figsize = (15,30), dpi = 80, facecolor ='w', edgecolor ='k')
plt.clf()
ax = fig.add_subplot(111)
ax.set_aspect(1)

plt.title("Tableau de contingence : grades nutritionnelle Vs groupe alimentaire N2", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Tableau de contingence : grades nutritionnelle Vs groupe alimentaire N2.jpg')
fig.savefig(path_complet, dpi=300, bbox_inches='tight')
res = sns.heatmap(df_crosstab, annot = False, fmt='.2f',cmap="YlGnBu", vmin=0.0, vmax =2053, linewidths=.5)
plt.show()

In [ ]:
# V de cramer
from scipy.stats.contingency import association
association(df_crosstab, method="cramer")

In [ ]:
#On remarque nous avons plus de de dependance entre les groupes alimentaires de niveau 2 avec le nutrtition grade
#qu'avec le groupe alimentaire 1

In [ ]:
#Verification qu il existe bien une corelation entre le score et le grade:
sns.set_theme()

fig, ax = plt.subplots(figsize=(20, 7))
ax = sns.boxplot(x='pnns_groups_2', y='nutrition-score-uk_100g', data=df_nutrition_grade)

plt.title("distribution du nutri-score par groupe alimentaire N2", fontsize=20, color='tab:blue', fontweight="bold")

plt.ylabel("Nutri-score", size = 20,color='tab:blue')
plt.xlabel("Groupe alimentaire", size = 15,color='tab:blue')
ax.tick_params(pad = 0, labelsize=15, colors='tab:purple')
plt.xticks(rotation = 90)
        
path_complet = os.path.join(graph_path,'distribution du score nutritionnel par groupe alimentaire N2.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')


plt.show()

# Analyse via les composants alimentaires

In [ ]:
df_float = df.select_dtypes(include = ['float64'])

In [ ]:
df_float.shape

In [ ]:
#Liste des 6 composants alimentaire
liste_composant = ['fat_100g','carbohydrates_100g','proteins_100g','vitamines_total','oligo_elem_total','sels_mineraux_total']


#liste des composants de la colonne 'fat_100g'
liste_lipide =['saturated-fat_100g','monounsaturated-fat_100g', 'polyunsaturated-fat_100g','trans-fat_100g', 
               'cholesterol_100g']

liste_satured_fat=['caprylic-acid_100g', 'capric-acid_100g', 'lauric-acid_100g','myristic-acid_100g',
                   'palmitic-acid_100g', 'stearic-acid_100g', 'arachidic-acid_100g','behenic-acid_100g', 
                   'montanic-acid_100g']

liste_monounsatured_fat=['omega-9-fat_100g','oleic-acid_100g', 'gondoic-acid_100g',]

liste_polyunsaturated_fat =['omega-3-fat_100g', 'alpha-linolenic-acid_100g', 'eicosapentaenoic-acid_100g',
                            'docosahexaenoic-acid_100g', 'omega-6-fat_100g', 'linoleic-acid_100g', 
                            'arachidonic-acid_100g', 'gamma-linolenic-acid_100g', 'dihomo-gamma-linolenic-acid_100g']


#liste des composant de la colonne "carbohydrates_100g"
liste_glucide = ['sugars_100g', 'sucrose_100g', 'glucose_100g', 'fructose_100g', 'lactose_100g', 'maltose_100g',
                 'maltodextrins_100g', 'starch_100g', 'polyols_100g', 'fiber_100g']

#liste des composant de la colonne "proteins_100g"
liste_proteines = ['casein_100g','serum-proteins_100g','nucleotides_100g']

#listes vitamine:
liste_vitamines = ['vitamin-a_100g', 'beta-carotene_100g', 'vitamin-d_100g', 'vitamin-e_100g', 'vitamin-k_100g', 
                   'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g', 'vitamin-pp_100g', 'vitamin-b6_100g', 
                   'vitamin-b9_100g', 'folates_100g', 'vitamin-b12_100g', 'biotin_100g', 'pantothenic-acid_100g']
#creation de la colonne "vitamines_total"
df_float['vitamines_total'] = df_float[liste_vitamines].sum(axis = 1, skipna=True)

#liste_sels_mineraux
liste_sels_mineraux = ['salt_100g','sodium_100g','calcium_100g','phosphorus_100g','magnesium_100g']
#creation de la colonne "sels_mineraux_total"
df_float['sels_mineraux_total'] = df_float[liste_sels_mineraux].sum(axis = 1, skipna=True)

#liste_oligo_elem
liste_oligo_elem = ['iron_100g','zinc_100g','copper_100g','manganese_100g', 'fluoride_100g','selenium_100g',
                    'chromium_100g', 'molybdenum_100g','iodine_100g']
#creation de la colonne "oligo_elem_total"
df_float['oligo_elem_total'] = df_float[liste_oligo_elem].sum(axis = 1, skipna=True)

In [ ]:
liste_composant.append('nutrition-score-uk_100g')

In [ ]:
corr_df =df_float[liste_composant].corr(method='pearson')
plt.figure(figsize=(10, 10))
cmap=sns.cubehelix_palette(20, light=0.95, dark=0.15)
sns.heatmap(corr_df, annot=True,cmap=cmap, linewidths=.5)
plt.title("Covariance entre composant nutritionnel et score nutritionnel", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'Covariance entre composant nutritionnel et score nutritionnel.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#il y a 3 composants qui semble avoir une incidance significative sur la nutrition_score:
#les lipides(Fat_100g),les glucides(Carbohydrates_100g), les proteines (proteins_100g).

In [ ]:
X = 'fat_100g'
Y = 'nutrition-score-uk_100g'
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0, 110)
ax.set_ylim(-20, 50)
sns.regplot(x=X ,y=Y, data=df, ax = ax, marker="+", line_kws={"color": "red"})

plt.title("linear regression nutri-score vs Fat", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'linear regression nutri-score vs Fat.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# linear regression coefs
from scipy import stats
x = df['fat_100g'].notna()
y = df['nutrition-score-uk_100g']
res = stats.linregress(x, y)
print(f"Y = {res.slope:.2f} * X + {res.intercept:.2f} \n")
print("test de significativité de la pente \n-H0 : la pente n'est pas signficatif \n-H1: La pente est sgnificative")
alpha = 0.05
if res.pvalue < alpha :
    print(f"-pvalue = {res.pvalue}")
    print(f"on rejette H0 au seuil {alpha*100}% , le coef directeur est significatif \n")
else:
    print(f"on ne rejette pas H0 au seuil de {alpha}, le coef directeur est null \n")
    
print(f"le coefficient de determination est : R-squared: {res.rvalue**2:.6f}")
print(f"Standard error of the estimated slope is : {res.stderr:.3f}")
print(f"Standard error of the estimated intercept is : {res.intercept_stderr:.3f}")

In [ ]:
X = 'carbohydrates_100g'
Y = 'nutrition-score-uk_100g'
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0, 110)
ax.set_ylim(-20, 50)
sns.regplot(x=X ,y=Y, data=df, ax = ax, marker="+", line_kws={"color": "red"})

plt.title("linear regression nutri-score vs carbohydrate", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'linear regression nutri-score vs carbohydrate.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# linear regression coefs
from scipy import stats
x = df['carbohydrates_100g'].notna()
y = df['nutrition-score-uk_100g']
res = stats.linregress(x, y)
print(f"Y = {res.slope:.2f} * X + {res.intercept:.2f} \n")
print("test de significativité de la pente \n-H0 : la pente n'est pas signficatif \n-H1: La pente est sgnificative")
alpha = 0.05
if res.pvalue < alpha :
    print(f"-pvalue = {res.pvalue}")
    print(f"on rejette H0 au seuil {alpha*100}% , le coef directeur est significatif \n")
else:
    print(f"on ne rejette pas H0 au seuil de {alpha}, le coef directeur est null \n")
    
print(f"le coefficient de determination est : R-squared: {res.rvalue**2:.6f}")
print(f"Standard error of the estimated slope is : {res.stderr:.3f}")
print(f"Standard error of the estimated intercept is : {res.intercept_stderr:.3f}")

In [ ]:
X = 'proteins_100g'
Y = 'nutrition-score-uk_100g'
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0, 110)
ax.set_ylim(-20, 50)
sns.regplot(x=X ,y=Y, data=df, ax = ax, marker="+", line_kws={"color": "red"})

plt.title("linear regression nutri-score vs proteins", fontsize=20, color='tab:blue', fontweight="bold")

path_complet = os.path.join(graph_path,'linear regression nutri-score vs proteins.jpg')
plt.savefig(path_complet, dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# linear regression coefs
from scipy import stats
x = df['proteins_100g'].notna()
y = df['nutrition-score-uk_100g']

res = stats.linregress(x, y)

print(f"Y = {res.slope:.2f} * X + {res.intercept:.2f} \n")
print("test de significativité de la pente \n-H0 : la pente n'est pas signficatif \n-H1: La pente est sgnificative")
alpha = 0.05
if res.pvalue < alpha :
    print(f"-pvalue = {res.pvalue}")
    print(f"on rejette H0 au seuil {alpha*100}% , le coef directeur est significatif \n")
else:
    print(f"on ne rejette pas H0 au seuil de {alpha}, le coef directeur est null \n")
    
print(f"le coefficient de determination est : R-squared: {res.rvalue**2:.6f}")
print(f"Standard error of the estimated slope is : {res.stderr:.3f}")
print(f"Standard error of the estimated intercept is : {res.intercept_stderr:.3f}")